In [ ]:
import cv2
import numpy as np
import serial, time
import tkinter as tk
from PIL import Image, ImageTk
from collections import deque

try:
    esp = serial.Serial('COM3', 115200, timeout=1)
    time.sleep(2)
except serial.SerialException:
    print("Error: Could not connect to ESP32 on COM3.")
    esp = None

color_ranges = [
    (np.array([125, 50, 70]), np.array([165, 255, 255]))
]
cap = cv2.VideoCapture(1)

root = tk.Tk()
root.title("Pink Object Tracker")
panel = tk.Label(root)
panel.pack()

positions = deque(maxlen=5)

def update_frame():
    ret, frame = cap.read()
    if not ret:
        root.after(20, update_frame)
        return

    frame = cv2.flip(frame, 1)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    h, w, _ = frame.shape
    cx = w // 2

    cv2.line(frame, (cx, 0), (cx, h), (0, 0, 255), 2)

    mask = sum([cv2.inRange(hsv, lower, upper) for lower, upper in color_ranges])
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        c = max(contours, key=cv2.contourArea)
        if cv2.contourArea(c) > 800:
            x, y, cw, ch = cv2.boundingRect(c)
            obj_x = x + cw // 2
            positions.append(obj_x)
            smooth_x = int(np.mean(positions))

            cv2.rectangle(frame, (x, y), (x + cw, y + ch), (0, 255, 0), 2)
            cv2.circle(frame, (smooth_x, y + ch // 2), 5, (255, 0, 0), -1)

            deadzone = 40
            if esp:
                if smooth_x < cx - deadzone:
                    esp.write(b"L")
                elif smooth_x > cx + deadzone:
                    esp.write(b"R")
                else:
                    esp.write(b"C")

    imgtk = ImageTk.PhotoImage(image=Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))
    panel.config(image=imgtk)
    panel.imgtk = imgtk

    root.after(30, update_frame)

def on_close():
    cap.release()
    if esp:
        esp.close()
    root.destroy()

root.protocol("WM_DELETE_WINDOW", on_close)
update_frame()
root.mainloop()